In [1]:
import pandas as pd

# File path
file_path = "/Users/zhouxiaoyu/Desktop/golem/cleaned_keywords_and_tags_V2_FP.csv"

# Read the CSV file
data = pd.read_csv(file_path)

# Display the first few rows
print(data.head())

         Keyword                                         Tag Values
0      character  reader-insert, voyeurism, emotional hurt/comfo...
1          event  misunderstandings, voyeurism, emotional hurt/c...
2  world setting  alternate universe - human, alpha/beta/omega d...
3      narrative  POV Alternating, POV Third Person, POV First P...
4       dialogue  Banter, No Dialogue, Dialogue-Only, Inner Dial...


In [2]:
import matplotlib.pyplot as plt
import pandas as pd
from SPARQLWrapper import SPARQLWrapper2, JSON

In [3]:
# 安装所需库
# 运行以下命令安装 SPARQLWrapper
# 在终端或 Jupyter Notebook 中运行：!pip install SPARQLWrapper
!pip install SPARQLWrapper

# 导入必要的模块
import matplotlib.pyplot as plt
import pandas as pd
from SPARQLWrapper import SPARQLWrapper2, JSON

# 设置SPARQL连接
sparql = SPARQLWrapper2("http://194.171.203.17:8890/sparql")
sparql.setReturnFormat(JSON)

# 定义执行查询并返回单一计数值的函数
def execute_count_query(query):
    sparql.setQuery(query)
    results = sparql.query().convert()
    count = int(results.bindings[0]['sCount'].value)
    return count

# 查询孤儿作品的总数量
orphan_total_count_query = """
PREFIX golem: <https://golemlab.eu/graph/>
SELECT (COUNT(?s) as ?sCount) WHERE {
    ?s golem:author_url "https://archiveofourown.org/users/orphan_account/pseuds/orphan_account" .
}
"""

# 执行查询并获取孤儿作品和非孤儿作品的总数量
orphan_total_count = execute_count_query(orphan_total_count_query)

# 打印总数量结果
print(f"Total FanfictionCount for Orphan Works: {orphan_total_count}")

Total FanfictionCount for Orphan Works: 276272


In [4]:
import pandas as pd

# 定义查询孤儿作品的前15种语言，并限制在 General Audiences 评级
orphan_language_query = """
PREFIX golem: <https://golemlab.eu/graph/>
SELECT ?language (COUNT(?s) as ?count) WHERE {
    ?s golem:author_url "https://archiveofourown.org/users/orphan_account/pseuds/orphan_account" .
    ?s golem:language ?language .
    ?s golem:rating "General Audiences" .
}
GROUP BY ?language
ORDER BY DESC(?count)
LIMIT 100
"""

# 执行查询并获取孤儿小说的前15种语言数据
sparql.setQuery(orphan_language_query)
orphan_results = sparql.query().convert()
orphan_languages = [(result['language'].value, int(result['count'].value)) for result in orphan_results.bindings]

# 转换为 DataFrame，并按数量降序排序
orphan_df = pd.DataFrame(orphan_languages, columns=['Language', 'Orphan Works Count']).sort_values(by='Orphan Works Count', ascending=False)


# 分别打印结果表格
print("Top 15 Languages in Orphan Works (General Audiences) (Sorted):\n", orphan_df)

Top 15 Languages in Orphan Works (General Audiences) (Sorted):
                 Language  Orphan Works Count
0                English               74559
1              中文-普通话 國語                 337
2                Español                 238
3                Русский                 175
4               Italiano                 123
5               Français                 103
6       Bahasa Indonesia                  77
7   Português brasileiro                  51
8                Deutsch                  38
9               Filipino                  29
10                Polski                  28
11                   한국어                  12
12            Tiếng Việt                  11
13           Slovenščina                   9
14               Svenska                   9
15            Українська                   7
16                Türkçe                   5
17                   日本語                   5
18                 Suomi                   3
19                 فارسی            

In [5]:
from SPARQLWrapper import SPARQLWrapper2, JSON
import pandas as pd

# 设置SPARQL连接
sparql = SPARQLWrapper2("http://194.171.203.17:8890/sparql")
sparql.setReturnFormat(JSON)

# 定义查询
query = """
PREFIX golem: <https://golemlab.eu/graph/>
SELECT DISTINCT ?story_id
WHERE {
    ?s golem:author_url "https://archiveofourown.org/users/orphan_account/pseuds/orphan_account" .
    ?s golem:rating ?rating .
    FILTER(?rating IN ("General Audiences", "Teen And Up Audiences")) .
    ?s golem:language ?language .
    FILTER(?language IN (
        "English", 
        "中文-普通话 國語", 
        "Español", 
        "Italiano", 
        "Bahasa Indonesia", 
        "Nederlands", 
        "한국어"
    )) .
    ?s golem:fandom ?fandom .
    FILTER(?fandom IN (
        "My Hero Academia", 
        "Bangtan Boys", 
        "Haikyuu!!", 
        "Supernatural", 
        "Harry Potter - J. K. Rowling", 
        "DCU", 
        "Marvel Cinematic Universe"
    )) .
    ?s golem:story_id ?story_id .
}
"""

# 执行查询并解析结果
def fetch_story_ids():
    sparql.setQuery(query)
    results = sparql.query().convert()
    
    # 打印 results.bindings 检查其结构（仅用于调试时）
    # print(results.bindings)
    
    # 提取 story_id 并保存为列表
    story_ids = [str(result['story_id']) for result in results.bindings]  # 转换为字符串
    return story_ids

# 获取故事 ID 列表
story_ids = fetch_story_ids()

# 将列表保存为 CSV 或打印输出
story_ids_df = pd.DataFrame(story_ids, columns=["story_id"])
story_ids_df.to_csv("story_ids.csv", index=False)  # 保存为CSV文件
print(story_ids_df)  # 打印结果

                              story_id
0      Value(typed-literal:'14202132')
1      Value(typed-literal:'31971964')
2      Value(typed-literal:'28724955')
3      Value(typed-literal:'28797012')
4      Value(typed-literal:'28895025')
...                                ...
31105   Value(typed-literal:'9045647')
31106   Value(typed-literal:'9840101')
31107   Value(typed-literal:'9862730')
31108  Value(typed-literal:'38596617')
31109   Value(typed-literal:'3871375')

[31110 rows x 1 columns]


In [5]:
from SPARQLWrapper import SPARQLWrapper2, JSON
import pandas as pd

# 设置SPARQL连接
sparql = SPARQLWrapper2("http://194.171.203.17:8890/sparql")
sparql.setReturnFormat(JSON)

# SPARQL 查询：General和Teen的10个作品的Tag
query_tags_sample = """
PREFIX golem: <https://golemlab.eu/graph/>
SELECT DISTINCT ?story_id ?keyword
WHERE {
    ?s golem:author_url "https://archiveofourown.org/users/orphan_account/pseuds/orphan_account" .
    ?s golem:rating ?rating .
    FILTER(?rating IN ("General Audiences", "Teen And Up Audiences")) .
    ?s golem:story_id ?story_id .
    ?s golem:keyword ?keyword .
}
LIMIT 10
"""

# 执行查询
sparql.setQuery(query_tags_sample)
results = sparql.query().convert()

# 提取数据到DataFrame
sample_data = [
    {"story_id": result["story_id"].value, "keyword": result["keyword"].value}
    for result in results.bindings
]
sample_df = pd.DataFrame(sample_data)

# 打印结果
if not sample_df.empty:
    print("Sample Data (10 Stories with Tags):")
    print(sample_df)
else:
    print("No data retrieved.")

Sample Data (10 Stories with Tags):
   story_id                                          keyword
0  28724955                       Slytherin Park Jimin (BTS)
1  28724955                         Gryffindor Jeon Jungkook
2  28724955  Established Kim Seokjin | Jin/Min Yoongi | Suga
3  28724955                       Ravenclaw Kim Namjoon | RM
4  28724955                   Namjoon wants to date Jungkook
5  28724955                        Park Jimin is an animagus
6  28724955                       Slytherin Kim Taehyung | V
7  28724955   Tae finds a cat that solves his lonely problem
8  28724955                                    Tae is lonely
9  28797012                                            Angst


In [6]:
import os
from SPARQLWrapper import SPARQLWrapper2, JSON
import pandas as pd

# 设置SPARQL连接
sparql = SPARQLWrapper2("http://194.171.203.17:8890/sparql")
sparql.setReturnFormat(JSON)

# 构造SPARQL查询
query_filtered_data = """
PREFIX golem: <https://golemlab.eu/graph/>
SELECT DISTINCT ?story_id ?language ?keyword ?fandom ?numberOfWords
WHERE {
    ?s golem:author_url "https://archiveofourown.org/users/orphan_account/pseuds/orphan_account" .
    ?s golem:rating ?rating .
    FILTER(?rating IN ("General Audiences", "Teen And Up Audiences")) .
    ?s golem:language ?language .
    FILTER(?language IN (
        "中文-普通话 國語", 
        "Español", 
        "Italiano", 
        "Bahasa Indonesia", 
        "Nederlands", 
        "한국어"
    )) .
    ?s golem:story_id ?story_id .
    ?s golem:keyword ?keyword .
    ?s golem:fandom ?fandom .
    ?s golem:numberOfWords ?numberOfWords .
}
"""

# 执行查询
sparql.setQuery(query_filtered_data)
results = sparql.query().convert()

# 提取数据到DataFrame
filtered_data = [
    {
        "story_id": result["story_id"].value,
        "language": result["language"].value,
        "keyword": result["keyword"].value,
        "fandom": result["fandom"].value,
        "word_count": int(result["numberOfWords"].value)  # 转换字数为整数
    }
    for result in results.bindings
]
filtered_df = pd.DataFrame(filtered_data)

# 创建一个字典来存储每种语言的DataFrame
language_dataframes = {}

# 检查是否有数据
if not filtered_df.empty:
    # 按语言分组并存储到字典中
    for language, group in filtered_df.groupby("language"):
        # 按 story_id 对关键词合并，并合并 fandom 和 word_count
        grouped_df = group.groupby("story_id").agg({
            "keyword": lambda x: ", ".join(x.unique()),  # 合并去重关键词
            "fandom": lambda x: ", ".join(x.unique()),   # 合并去重 fandom
            "word_count": "first"  # 假设每个 story_id 对应唯一 word_count
        }).reset_index()
        
        # 存储到字典
        language_dataframes[language] = grouped_df
        
        # 打印语言、前几行和数量
        print(f"Language: {language}")
        print(grouped_df.head())
        print(f"Total stories: {grouped_df.shape[0]}\n")
else:
    print("No data retrieved.")

Language: Bahasa Indonesia
   story_id                                            keyword  \
0  10098365  Alternate Universe - Coffee Shops & Cafés, Hum...   
1   1018593                            Angst, Romance, Songfic   
2   1018638         Angst, POV Second Person, Romance, Songfic   
3  10394925  Domestic Fluff, Fluff, GL, Romance, Translatio...   
4  10436208  Alternate Universe, Angst, Hallucinations, Sib...   

               fandom  word_count  
0  Joker Game (Anime)        1657  
1          Durarara!!        1172  
2          Durarara!!        1025  
3      Tawur (Comics)        2346  
4  BoBoiBoy (Cartoon)         553  
Total stories: 183

Language: Español
   story_id                                            keyword  \
0   1018189                             From Sex to Love, Nosh   
1  10224620  First Kiss, Fluff, Sharing a Bed, No Spoilers,...   
2  10358214  Alternate Universe - College/University, Aroma...   
3  10481418  Alternate Universe - Coffee Shops & Cafés, Al

In [8]:
import os

# 文件保存目录
output_directory = "/Users/zhouxiaoyu/Desktop/golem/id_v2"
os.makedirs(output_directory, exist_ok=True)  # 如果文件夹不存在则创建

# 检查数据并保存每种语言的数据
if not filtered_df.empty:
    for language, df in language_dataframes.items():
        # 定义保存文件路径
        sanitized_language = language.replace(" ", "_").replace("/", "_")  # 确保文件名合法
        output_file_path = os.path.join(output_directory, f"{sanitized_language}.csv")
        
        # 保存 CSV 文件
        df.to_csv(output_file_path, index=False)
        print(f"Saved {language} data to {output_file_path}")
else:
    print("No data to save.")

Saved Bahasa Indonesia data to /Users/zhouxiaoyu/Desktop/golem/id_v2/Bahasa_Indonesia.csv
Saved Español data to /Users/zhouxiaoyu/Desktop/golem/id_v2/Español.csv
Saved Italiano data to /Users/zhouxiaoyu/Desktop/golem/id_v2/Italiano.csv
Saved Nederlands data to /Users/zhouxiaoyu/Desktop/golem/id_v2/Nederlands.csv
Saved 中文-普通话 國語 data to /Users/zhouxiaoyu/Desktop/golem/id_v2/中文-普通话_國語.csv
Saved 한국어 data to /Users/zhouxiaoyu/Desktop/golem/id_v2/한국어.csv


In [9]:
import os
from SPARQLWrapper import SPARQLWrapper2, JSON
import pandas as pd

# 设置SPARQL连接
sparql = SPARQLWrapper2("http://194.171.203.17:8890/sparql")
sparql.setReturnFormat(JSON)

# 构造SPARQL查询
query_filtered_data = """
PREFIX golem: <https://golemlab.eu/graph/>
SELECT DISTINCT ?story_id ?language ?keyword ?fandom ?numberOfWords
WHERE {
    ?s golem:author_url "https://archiveofourown.org/users/orphan_account/pseuds/orphan_account" .
    ?s golem:rating ?rating .
    FILTER(?rating IN ("General Audiences", "Teen And Up Audiences")) .
    ?s golem:language ?language .
    FILTER(?language = "English") .  
    ?s golem:fandom ?fandom .
    FILTER(?fandom IN (
        "My Hero Academia", 
        "Bangtan Boys", 
        "Haikyuu!!", 
        "Supernatural", 
        "Harry Potter - J. K. Rowling"
    )) .
    ?s golem:story_id ?story_id .
    ?s golem:keyword ?keyword .
    ?s golem:numberOfWords ?numberOfWords .
}
"""

# 执行查询
sparql.setQuery(query_filtered_data)
results = sparql.query().convert()

# 提取数据到DataFrame
filtered_data = [
    {
        "story_id": result["story_id"].value,
        "language": result["language"].value,
        "keyword": result["keyword"].value,
        "fandom": result["fandom"].value,
        "word_count": int(result["numberOfWords"].value)  # 转换字数为整数
    }
    for result in results.bindings
]
filtered_df = pd.DataFrame(filtered_data)

# 检查是否有数据
if not filtered_df.empty:
    # 按 story_id 对关键词合并，并保留 fandom 和 word_count
    grouped_df = filtered_df.groupby("story_id").agg({
        "language": "first",
        "keyword": lambda x: ", ".join(x),
        "fandom": "first",  # 假设每个 story_id 对应唯一 fandom
        "word_count": "first"  # 假设每个 story_id 对应唯一 word_count
    }).reset_index()
    
    # 打印前几行和数量
    print(f"Language: English with restricted fandom")
    print(grouped_df.head())
    print(f"Total stories: {grouped_df.shape[0]}\n")
else:
    print("No data retrieved.")

Language: English with restricted fandom
   story_id language                                            keyword  \
0  10012844  English                                Yoonseok - Freeform   
1  10023968  English     Fluff, Gay Chicken, Party Games, Kissing Games   
2  10029035  English                              Angst, Slash, Tragedy   
3  10034099  English  Book 5: Harry Potter and the Order of the Phoe...   
4  10055831  English  Fluff, lots and lots of fluff, Kittens, really...   

                         fandom  word_count  
0                  Bangtan Boys        1569  
1  Harry Potter - J. K. Rowling        1807  
2  Harry Potter - J. K. Rowling        1147  
3  Harry Potter - J. K. Rowling        2120  
4  Harry Potter - J. K. Rowling         163  
Total stories: 4198



In [10]:
import os

# 定义保存目录和文件名
output_directory = "/Users/zhouxiaoyu/Desktop/golem/id_v2"
os.makedirs(output_directory, exist_ok=True)  # 确保目标目录存在

output_file_path = os.path.join(output_directory, "English.csv")

# 检查数据并保存
if not filtered_df.empty:
    # 保存已分组的数据为 CSV
    grouped_df.to_csv(output_file_path, index=False)
    print(f"Saved English data to {output_file_path}")
else:
    print("No English data to save.")

Saved English data to /Users/zhouxiaoyu/Desktop/golem/id_v2/English.csv


In [14]:
import pandas as pd

# 文件路径
file_path = "/Users/zhouxiaoyu/Desktop/golem/id_v2/Bahasa_Indonesia.csv"

# 加载文件并打印部分内容
try:
    # 读取 CSV 文件
    bahasa_data = pd.read_csv(file_path)
    
    # 打印列名
    print("Columns in Bahasa Indonesia file:")
    print(bahasa_data.columns)
    
    # 打印前几行
    print("\nFirst few rows in Bahasa Indonesia file:")
    print(bahasa_data.head())
    
    # 检查数据基本信息
    print("\nSummary of the data:")
    print(bahasa_data.info())

except Exception as e:
    print(f"Error reading file: {e}")

Columns in Bahasa Indonesia file:
Index(['story_id', 'keywords', 'fandoms', 'word_count'], dtype='object')

First few rows in Bahasa Indonesia file:
   story_id                                           keywords  \
0  10098365  Alternate Universe - Coffee Shops & Cafés, Bus...   
1   1018593                            Angst, Romance, Songfic   
2   1018638         Angst, POV Second Person, Romance, Songfic   
3  10394925  Domestic Fluff, Fluff, GL, Love Confession, Ro...   
4  10436208  Alternate Universe, Angst, Hallucinations, Kai...   

              fandoms  word_count  
0  Joker Game (Anime)        1657  
1          Durarara!!        1172  
2          Durarara!!        1025  
3      Tawur (Comics)        2346  
4  BoBoiBoy (Cartoon)         553  

Summary of the data:
<class 'pandas.core.frame.DataFrame'>
RangeIndex: 183 entries, 0 to 182
Data columns (total 4 columns):
 #   Column      Non-Null Count  Dtype 
---  ------      --------------  ----- 
 0   story_id    183 non-null   

In [15]:
for language, file_path in language_files.items():
    try:
        print(f"Processing {language}...")
        # 加载语言数据
        language_data = pd.read_csv(file_path)

        # 打印列名和前几行用于调试
        print(f"Columns in {language} file: {language_data.columns}")
        print(f"First few rows in {language} file:\n{language_data.head()}\n")

        # 检查是否存在 keyword 或其变体列
        if 'keyword' not in language_data.columns and 'keywords' in language_data.columns:
            print(f"Renaming 'keywords' to 'keyword' for {language}")
            language_data.rename(columns={'keywords': 'keyword'}, inplace=True)

        # 清理关键词列
        language_data['keyword'] = clean_and_split_column(language_data['keyword'])

        # 添加新列 tag_list
        language_data['tag_list'] = language_data['keyword'].apply(exact_match_tags)

        # 删除 tag_list 为空的行
        language_data = language_data[language_data['tag_list'].str.len() > 0]

        # 添加 category 列
        def get_categories(tag_list):
            return list({tag_to_category[tag] for tag in tag_list})  # 去重后返回

        language_data['category'] = language_data['tag_list'].apply(get_categories)

        # 提取最终字段
        result_df = language_data[['story_id', 'tag_list', 'category', 'fandoms', 'word_count']]

        # 按 category 数量和字母顺序排序
        result_df = result_df.sort_values(
            by=['category', 'story_id'],
            key=lambda x: x if x.name != 'category' else x.apply(len),
            ascending=[False, True]
        )

        # 重新排列 category 内的值按字母顺序
        result_df['category'] = result_df['category'].apply(sorted)

        # 存储处理后的 DataFrame
        processed_dataframes[language] = result_df

        # 打印前几行供检查
        print(f"\nProcessed Data for {language}:")
        print(result_df.head())
    except Exception as e:
        print(f"Error processing {language}: {e}")

Processing Bahasa_Indonesia...
Columns in Bahasa_Indonesia file: Index(['story_id', 'keywords', 'fandoms', 'word_count'], dtype='object')
First few rows in Bahasa_Indonesia file:
   story_id                                           keywords  \
0  10098365  Alternate Universe - Coffee Shops & Cafés, Bus...   
1   1018593                            Angst, Romance, Songfic   
2   1018638         Angst, POV Second Person, Romance, Songfic   
3  10394925  Domestic Fluff, Fluff, GL, Love Confession, Ro...   
4  10436208  Alternate Universe, Angst, Hallucinations, Kai...   

              fandoms  word_count  
0  Joker Game (Anime)        1657  
1          Durarara!!        1172  
2          Durarara!!        1025  
3      Tawur (Comics)        2346  
4  BoBoiBoy (Cartoon)         553  

Renaming 'keywords' to 'keyword' for Bahasa_Indonesia

Processed Data for Bahasa_Indonesia:
     story_id                                           tag_list  \
67   22291090  [alternate universe  modern sett

In [16]:
import os

# 定义保存路径
output_directory = "/Users/zhouxiaoyu/Desktop/golem/id_v2"
os.makedirs(output_directory, exist_ok=True)  # 确保目标目录存在

# 保存每种语言的处理结果
for language, df in processed_dataframes.items():
    # 定义文件名和保存路径
    output_file_path = os.path.join(output_directory, f"{language}2.csv")
    
    # 保存为 CSV
    df.to_csv(output_file_path, index=False)
    print(f"Saved processed data for {language} to {output_file_path}")

Saved processed data for English to /Users/zhouxiaoyu/Desktop/golem/id_v2/English2.csv
Saved processed data for Bahasa_Indonesia to /Users/zhouxiaoyu/Desktop/golem/id_v2/Bahasa_Indonesia2.csv
Saved processed data for Español to /Users/zhouxiaoyu/Desktop/golem/id_v2/Español2.csv
Saved processed data for Italiano to /Users/zhouxiaoyu/Desktop/golem/id_v2/Italiano2.csv
Saved processed data for Nederlands to /Users/zhouxiaoyu/Desktop/golem/id_v2/Nederlands2.csv
Saved processed data for 中文-普通话_國語 to /Users/zhouxiaoyu/Desktop/golem/id_v2/中文-普通话_國語2.csv
Saved processed data for 한국어 to /Users/zhouxiaoyu/Desktop/golem/id_v2/한국어2.csv


In [17]:
!pip install selenium
!pip install webdriver-manager
!pip install beautifulsoup4 requests

In [18]:
from selenium import webdriver
from selenium.webdriver.common.by import By
from selenium.webdriver.support.ui import WebDriverWait
from selenium.webdriver.support import expected_conditions as EC
from selenium.webdriver.chrome.options import Options
from selenium.webdriver.common.action_chains import ActionChains
from selenium.webdriver.chrome.service import Service as ChromeService
from webdriver_manager.chrome import ChromeDriverManager
from bs4 import BeautifulSoup
import requests
import random
import time
import pandas as pd
from getpass import getpass
import json

In [19]:
import pandas as pd
import os

# 定义文件路径
language_files = [
    "/Users/zhouxiaoyu/Desktop/golem/id_v2/Bahasa_Indonesia2.csv",
    "/Users/zhouxiaoyu/Desktop/golem/id_v2/Español2.csv",
    "/Users/zhouxiaoyu/Desktop/golem/id_v2/Italiano2.csv",
    "/Users/zhouxiaoyu/Desktop/golem/id_v2/Nederlands2.csv",
    "/Users/zhouxiaoyu/Desktop/golem/id_v2/中文-普通话_國語2.csv",
    "/Users/zhouxiaoyu/Desktop/golem/id_v2/한국어2.csv",
    "/Users/zhouxiaoyu/Desktop/golem/id_v2/English2.csv",
]

# 基础 URL
base_url = "https://archiveofourown.org/works/"

# 保存路径
output_directory = "/Users/zhouxiaoyu/Desktop/golem/id_v2"
os.makedirs(output_directory, exist_ok=True)

# 批量处理
for file_path in language_files:
    # 提取语言名称
    language_name = os.path.basename(file_path).replace("2.csv", "")
    print(f"Processing {language_name}...")

    # 加载文件为 DataFrame
    df = pd.read_csv(file_path)
    
    # 检查是否包含 story_id 列
    if "story_id" not in df.columns:
        print(f"Error: 'story_id' column missing in {language_name}")
        continue

    # 生成 URL 列
    df["url"] = df["story_id"].apply(lambda story_id: f"{base_url}{story_id}?view_full_work=true")
    
    # 保存带 URL 的新文件
    output_file_path = os.path.join(output_directory, f"{language_name}_with_urls.csv")
    df.to_csv(output_file_path, index=False)
    print(f"Saved {language_name} with URLs to {output_file_path}")

Processing Bahasa_Indonesia...
Saved Bahasa_Indonesia with URLs to /Users/zhouxiaoyu/Desktop/golem/id_v2/Bahasa_Indonesia_with_urls.csv
Processing Español...
Saved Español with URLs to /Users/zhouxiaoyu/Desktop/golem/id_v2/Español_with_urls.csv
Processing Italiano...
Saved Italiano with URLs to /Users/zhouxiaoyu/Desktop/golem/id_v2/Italiano_with_urls.csv
Processing Nederlands...
Saved Nederlands with URLs to /Users/zhouxiaoyu/Desktop/golem/id_v2/Nederlands_with_urls.csv
Processing 中文-普通话_國語...
Saved 中文-普通话_國語 with URLs to /Users/zhouxiaoyu/Desktop/golem/id_v2/中文-普通话_國語_with_urls.csv
Processing 한국어...
Saved 한국어 with URLs to /Users/zhouxiaoyu/Desktop/golem/id_v2/한국어_with_urls.csv
Processing English...
Saved English with URLs to /Users/zhouxiaoyu/Desktop/golem/id_v2/English_with_urls.csv


In [21]:
import pandas as pd

# 文件路径
file_path = "/Users/zhouxiaoyu/Desktop/golem/id_v2/Bahasa_Indonesia_with_urls.csv"

# 加载 CSV 文件并命名为 source_data
try:
    source_data = pd.read_csv(file_path)
    
    # 打印 DataFrame 的基本信息
    print("source_data Loaded Successfully!")
    print(f"\nColumns in source_data:\n{source_data.columns}")
    print("\nFirst few rows of source_data:")
    print(source_data.head())
    
    # 打印数据类型和摘要
    print("\nsource_data Info:")
    print(source_data.info())
except Exception as e:
    print(f"Error loading file: {e}")

source_data Loaded Successfully!

Columns in source_data:
Index(['story_id', 'tag_list', 'category', 'fandoms', 'word_count', 'url'], dtype='object')

First few rows of source_data:
   story_id                                           tag_list  \
0  22291090  ['alternate universe  modern setting', 'family...   
1  27987429  ['alternate universe  high school', 'cuddling ...   
2  27988029  ['alternate universe  high school', 'cuddling ...   
3   1284895  ['alternate universe  collegeuniversity', 'hur...   
4   4047175                                ['crack', 'family']   

                                      category  \
0  ['character', 'narrative', 'world setting']   
1      ['character', 'event', 'world setting']   
2      ['character', 'event', 'world setting']   
3                   ['event', 'world setting']   
4                       ['character', 'event']   

                                 fandoms  word_count  \
0                  Shirah Nabawiyyah RPF         372   
1       

In [22]:
source_data.head()

,story_id,tag_list,category,fandoms,word_count,url
0,22291090,"['alternate universe modern setting', 'family...","['character', 'narrative', 'world setting']",Shirah Nabawiyyah RPF,372,https://archiveofourown.org/works/22291090?vie...
1,27987429,"['alternate universe high school', 'cuddling ...","['character', 'event', 'world setting']","BTS, Bangtan Boys, 방탄소년단",1739,https://archiveofourown.org/works/27987429?vie...
2,27988029,"['alternate universe high school', 'cuddling ...","['character', 'event', 'world setting']","BTS, Bangtan Boys, 방탄소년단",1739,https://archiveofourown.org/works/27988029?vie...
3,1284895,"['alternate universe collegeuniversity', 'hur...","['event', 'world setting']","Kuroko no Basuke, Kuroko's Basketball",2386,https://archiveofourown.org/works/1284895?view...
4,4047175,"['crack', 'family']","['character', 'event']","Football RPF, Sports RPF",267,https://archiveofourown.org/works/4047175?view...


In [36]:
def login(driver, username, password):
    driver.get("https://archiveofourown.org/") 
    
    try:
        # Click the login dropdown
        login_dropdown = WebDriverWait(driver, 10).until(EC.element_to_be_clickable((By.ID, 'login-dropdown')))
        login_dropdown.click()
        
        # Enter username
        username_field = WebDriverWait(driver, 10).until(EC.presence_of_element_located((By.ID, 'user_session_login_small')))
        username_field.send_keys(username)
        
        # Enter password
        password_field = WebDriverWait(driver, 10).until(EC.presence_of_element_located((By.ID, 'user_session_password_small')))
        password_field.send_keys(password)
        
        # Click login button
        login_button = WebDriverWait(driver, 10).until(EC.element_to_be_clickable((By.NAME, 'commit')))
        login_button.click()
        
        print("Logged in successfully")
    except Exception as e:
        print("Error during login:", e)
        driver.quit()

# For some works you need to be logged in, maybe not needed for orphaned works, so you'll have to modify the code a bit
# Login
username = "Xiaoyu_GolemLab"
password = "ZXYmm123"

In [37]:
# Define sleep time to prevent banning from server
def sleep():
    sleep_time = 3 + random.randint(0, 10) + random.random()
    time.sleep(sleep_time)

# List of URLs to scrape
urls = source_data.url.to_list()

# Set up Chrome options
options = Options()
options.add_argument("--headless")
options.add_argument("--disable-gpu")

# Sample user agents to rotate between requests
user_agents = [
    "Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/91.0.4472.124 Safari/537.36",
    "Mozilla/5.0 (Macintosh; Intel Mac OS X 10_15_7) AppleWebKit/605.1.15 (KHTML, like Gecko) Version/14.0 Safari/605.1.15",
    "Mozilla/5.0 (iPhone; CPU iPhone OS 13_5 like Mac OS X) AppleWebKit/605.1.15 (KHTML, like Gecko) Version/13.0 Mobile/15E148 Safari/604.1",
    "Mozilla/5.0 (Windows NT 10.0; Win64; x64; rv:89.0) Gecko/20100101 Firefox/89.0"
]

def setup_driver(user_agent):
    """Set up WebDriver with a specified user agent."""
    options = webdriver.ChromeOptions()
    options.add_argument(f"user-agent={user_agent}")
    return webdriver.Chrome(service=ChromeService(ChromeDriverManager().install()), options=options)

In [38]:
def extract_data_from_page(url):
    try:
         driver.get(url)

        # retrieve the page source for parsing
         page_source = driver.page_source
    
        # Use BeautifulSoup to parse the page
         soup = BeautifulSoup(page_source, 'html.parser')

        # Locate the main section with meta information
         meta_section = soup.find("dl", class_="work meta group")
         if not meta_section:
            print(f"Meta section not found on page: {url}")
            return None

        # Extract Language
         language = meta_section.find("dd", class_="language")
         language_text = language.get_text(strip=True) if language else "N/A"

        # Extract Stats (Published, Updated, Words, etc.)
         stats_section = meta_section.find("dd", class_="stats")
         stats_data = {}
         if stats_section:
            # Iterate over each stat type
            for stat in stats_section.find_all("dt"):
                stat_type = stat.get_text(strip=True).rstrip(":")
                stat_value = stat.find_next_sibling("dd").get_text(strip=True)
                stats_data[stat_type] = stat_value
         else:
             print(f"Stats section not found on page: {url}")
        
        # Organize and return the extracted data
         data = {
            "URL": url,
            "Language": language_text,
            "Stats": stats_data,
        }
        
    except Exception as e:
        print(f"Error: {e}")
        print(f"Skipping URL: {url}")

    sleep() 
    
    return data

In [39]:
# Choose a random user agent for each URL
user_agent = random.choice(user_agents)
print(f"Using User Agent: {user_agent}")

# Initialize the driver with the chosen user agent
driver = setup_driver(user_agent)

# Login
login(driver, username, password)

# Iterate over URLs and scrape data
all_data = []
for url in urls:
    page_data = extract_data_from_page(url)
    if page_data:
        all_data.append(page_data)

Using User Agent: Mozilla/5.0 (Macintosh; Intel Mac OS X 10_15_7) AppleWebKit/605.1.15 (KHTML, like Gecko) Version/14.0 Safari/605.1.15
Logged in successfully


In [40]:
for entry in all_data:
    print(entry)

{'URL': 'https://archiveofourown.org/works/22291090?view_full_work=true', 'Language': 'Bahasa Indonesia', 'Stats': {'Published': '2020-01-17', 'Updated': '2020-01-17', 'Words': '372', 'Chapters': '1/3', 'Kudos': '1', 'Hits': '163'}}
{'URL': 'https://archiveofourown.org/works/27987429?view_full_work=true', 'Language': 'Bahasa Indonesia', 'Stats': {'Published': '2020-12-10', 'Words': '1,739', 'Chapters': '1/1', 'Kudos': '1', 'Bookmarks': '2', 'Hits': '27'}}
{'URL': 'https://archiveofourown.org/works/27988029?view_full_work=true', 'Language': 'Bahasa Indonesia', 'Stats': {'Published': '2020-12-10', 'Words': '1,739', 'Chapters': '1/1', 'Bookmarks': '1', 'Hits': '63'}}
{'URL': 'https://archiveofourown.org/works/1284895?view_full_work=true', 'Language': 'Bahasa Indonesia', 'Stats': {'Published': '2014-03-08', 'Words': '2,386', 'Chapters': '1/1', 'Kudos': '13', 'Bookmarks': '1', 'Hits': '681'}}
{'URL': 'https://archiveofourown.org/works/4047175?view_full_work=true', 'Language': 'Bahasa Indone

In [47]:
import pandas as pd
import numpy as np

# 为 source_data 添加空列
source_data['kudos'] = np.nan
source_data['hits'] = np.nan

# 更新 kudos 和 hits 数据到 source_data
for entry in all_data:
    url = entry['URL']
    kudos = entry['Stats'].get('Kudos', 'N/A')  # 获取 Kudos，若不存在则填 'N/A'
    hits = entry['Stats'].get('Hits', 'N/A')    # 获取 Hits，若不存在则填 'N/A'
    
    # 找到对应 URL 的行并更新
    source_data.loc[source_data['url'] == url, 'kudos'] = kudos
    source_data.loc[source_data['url'] == url, 'hits'] = hits

# 查看更新后的数据
print(source_data)

    story_id                                           tag_list  \
0   22291090  ['alternate universe  modern setting', 'family...   
1   27987429  ['alternate universe  high school', 'cuddling ...   
2   27988029  ['alternate universe  high school', 'cuddling ...   
3    1284895  ['alternate universe  collegeuniversity', 'hur...   
4    4047175                                ['crack', 'family']   
..       ...                                                ...   
66  28362273                                      ['slow burn']   
67  31000823                                 ['age difference']   
68  33009682          ['alternate universe  collegeuniversity']   
69  33397867  ['alternate universe  modern setting', 'domest...   
70  37559890                               ['love confessions']   

                                       category  \
0   ['character', 'narrative', 'world setting']   
1       ['character', 'event', 'world setting']   
2       ['character', 'event', 'world sett